## 講習3 --- 星の測光  

点光源の明るさを測定してみます。ここでは、IRAFのAPPHOTを用いて、アパーチャ測光を行います。  


### 星をひとつだけ選んで測光してみる  

講習2で処理をした、idf0011.fitsの真ん中あたりの星の明るさを測ってみましょう。  

#### 準備 
- 星の位置を求めておく
- 星のサイズ(fwhm)を求めておく
- 背景のレベルとばらつきをもとめておく

In [1]:
from pyraf import iraf

In [2]:
iraf.display('idf0011.fits')

z1=-57.03943 z2=123.6036


In [3]:
iraf.imexam()

#   COL    LINE   COORDINATES
#     R    MAG    FLUX     SKY    PEAK    E   PA BETA ENCLOSED   MOFFAT DIRECT
 515.88  508.70 515.88 508.70
   8.15  12.54  96499.   4.009   8887. 0.15   45 3.54     2.74     2.79   2.72
#            SECTION     NPIX     MEAN   MEDIAN   STDDEV      MIN      MAX
   [549:553,527:531]       25    3.369       0.    13.86   -23.69     35.5
   [507:511,540:544]       25    11.88    14.34    20.27     -38.    54.91
   [487:491,518:522]       25    8.139    7.602    21.01   -47.66    42.74
   [495:499,486:490]       25    4.095    2.161     20.3   -44.92    47.67
   [530:534,468:472]       25    1.239    1.014    21.19   -52.36    36.89
   [543:547,493:497]       25     6.26    5.085    20.16   -32.92    49.59


星の位置は(x, y) = (516, 509)で、fwhm=2.8、背景のレベルとばらつきは、0と20としておきましょう。 

#### パラメータの設定  

apphotモジュールをimportします。

In [4]:
from iraf import apphot

~/iraf/login.cl の記載内容によっては、
``` python 
from iraf import noao
from iraf import digiphot
``` 
も必要かもしれません。 

次に、測光に必要なパラメータを設定します。  

In [5]:
med = 0.  #  背景レベルとばらつき、fwhm
std = 20.
fwhm = 2.8

iraf.apphot.unlearn()  #  デフォルト値に戻しておく

iraf.apphot.datapars.datamax = 20000  #  検出器に特有な値
iraf.apphot.datapars.readnoise = 30 #  検出器に特有な値
iraf.apphot.datapars.epadu = 5 #  検出器に特有な値
iraf.apphot.datapars.itime = 5  # 積分時間 

iraf.apphot.phot.interactive = 'no'  
iraf.apphot.phot.verify = 'no'
iraf.apphot.phot.verbose = 'no'
iraf.apphot.photpars.zmag = 21  # 等級のゼロ点

iraf.apphot.datapars.fwhmpsf = fwhm   #  fwhmで決まるパラメータ
iraf.apphot.centerpars.cbox = max(5.0, fwhm)
iraf.apphot.fitskypars.annulus = 3 * fwhm
iraf.apphot.photpars.apertures = 3 * fwhm
iraf.apphot.fitskypars.dannulus = 5.

iraf.apphot.datapars.sigma = std  #  背景のレベルとばらつきで決まるパラメータ
iraf.apphot.datapars.datamin = med - 5 * std

星のおよその位置をテキストファイルに書き込んでおきます。これはテキストエディタで手動で書き込んでおきましょう。

In [6]:
cat coo.txt   #  catはファイルの中身を見るlinuxコマンド 

516 509


この座標ファイルを、**iraf.phot()**に読み込ませます。

In [7]:
iraf.phot('idf0011.fits', coords='coo.txt', output='out.mag')

最初の引数は、測光する対象のFITSファイル名です。coords=で読み込ませる座標ファイルの名前、output=で結果を書き出すファイル名を指定します。  

結果のファイルは、iraf.phot()固有の形式で書き出されています。

In [8]:
#cat 'out.mag'   # githubでは長くなるのでコメントアウトしておきます

通常、必要なのは、xcenter, ycenter, mag, merr です。**iraf.txdump()**を使って、それらだけを抜き出します。

In [9]:
iraf.txdump('out.mag', fields='xc,yc,mag,merr')

515.882  508.629  10.294  0.005


Stdout=でファイル名を指定すれば、そこに書き出してくれます。

In [10]:
iraf.txdump('out.mag', fields='xc,yc,mag,merr', Stdout='result.txt')

これで、ひとつだけ選んだ星の機械等級がもとまりました。  

### 視野の星を検出させて測光してみる

この視野の他の星の明るさも測りたいときには、どうするのでしょうか。まさか、全ての星の位置について手動で座標ファイルを作成するわけにはいきません。**iraf.daofind()**を使います。背景の'ばらつき'よりもどのくらい明るい星だけを見つけろ、という指示を与えます。  

上の測光に必要なパラメータ設定に加えて、次も設定してやります。

In [11]:
iraf.apphot.findpars.threshold = 10  #  10シグマ以上のものを検出せよ 
iraf.apphot.findpars.sharphi = 0.8  # 星っぽくないものを除くため

iraf.apphot.daofind.output = 'out.coo'  #  daofindの出力には、よく.cooという拡張子が使われる
iraf.apphot.daofind.interac = 'no'
iraf.apphot.daofind.verify = 'no'

In [12]:
iraf.daofind('idf0011.fits')

In [13]:
#cat 'out.coo'  # githubでは長くなるのでコメントアウトしておきます

40個くらい星が検出されたようです。  
先ほどと同様に、このファイルをiraf.phot()のcoords=で指定してやり、今度は結果をout2.magに出しましょう。

In [14]:
iraf.phot('idf0011.fits', coords='out.coo', output='out2.mag')

In [15]:
iraf.txdump('out2.mag', fields='xc,yc,mag,merr')

312.002  30.900  13.578  0.086
785.511  63.498  11.548  0.014
250.494  113.489  11.472  0.012
603.316  119.958  12.481  0.033
222.467  129.443  12.633  0.036
713.868  188.083  13.131  0.057
502.051  207.508  12.914  0.045
174.419  237.868  13.524  0.087
35.204  238.805  11.930  0.020
673.523  250.965  13.671  0.094
598.713  265.580  13.986  0.119
646.487  270.673  11.053  0.008
239.394  299.813  INDEF  INDEF
239.506  303.484  INDEF  INDEF
288.244  341.014  13.686  0.096
295.074  357.658  12.376  0.031
753.546  379.476  13.308  0.065
459.538  411.408  14.010  0.120
94.773  455.272  13.830  0.105
940.446  461.984  11.477  0.012
515.882  508.629  10.294  0.005
355.176  583.665  12.615  0.037
952.449  599.439  13.476  0.071
389.024  681.497  13.643  0.090
52.301  703.531  9.513  0.003
748.465  705.381  12.487  0.033
235.315  705.592  12.621  0.037
52.876  708.867  9.492  0.003
425.532  781.090  INDEF  INDEF
421.137  786.516  INDEF  INDEF
427.249  787.271  INDEF  INDEF
429.702  792.182  IND

INDEFなんてのもあります。これは、たまたまバッドピクセルが測光領域に含まれていた、あるいは、サチった星です。  
測光誤差を0.1等以下のものだけに絞ることで、INDEFのものも削除できます。  

In [16]:
iraf.txdump('out2.mag', fields='xc,yc,mag,merr', expr='merr < 0.1', Stdout='result2.txt')

In [17]:
!head -n 10 result2.txt   #  一般的なlinuxコマンドは!をつけて実行可能 

312.002  30.900  13.578  0.086
785.511  63.498  11.548  0.014
250.494  113.489  11.472  0.012
603.316  119.958  12.481  0.033
222.467  129.443  12.633  0.036
713.868  188.083  13.131  0.057
502.051  207.508  12.914  0.045
174.419  237.868  13.524  0.087
35.204  238.805  11.930  0.020
673.523  250.965  13.671  0.094


### 測光値の較正  

ここで求まったものは、機械等級です。等級のゼロ点を適当に設定したものです。 
どうやって本当の等級に直せば良いでしょうか。  

実は、target1の真ん中の星は標準星といって、明るさのわかっている星です。  
Persson et al. (1998)のNo.9182の星で、Hバンドでの等級は H=11.082(0.010)です。  

この標準星の機械等級が10.294と求められたので、

In [18]:
11.082 - 10.294

0.7880000000000003

result2.txtの全ての星の等級に + 0.788足せば較正ができたことになります。  

In [19]:
import numpy as np
np.set_printoptions(precision=3, suppress=True)  #  suppress=Trueで指数表示禁止

In [20]:
mlist = np.loadtxt('result2.txt')
mlist[:, 2] = mlist[:, 2] + 0.788   #  3列目全てに0.788を足す 
print (mlist)

[[  312.002    30.9      14.366     0.086]
 [  785.511    63.498    12.336     0.014]
 [  250.494   113.489    12.26      0.012]
 [  603.316   119.958    13.269     0.033]
 [  222.467   129.443    13.421     0.036]
 [  713.868   188.083    13.919     0.057]
 [  502.051   207.508    13.702     0.045]
 [  174.419   237.868    14.312     0.087]
 [   35.204   238.805    12.718     0.02 ]
 [  673.523   250.965    14.459     0.094]
 [  646.487   270.673    11.841     0.008]
 [  288.244   341.014    14.474     0.096]
 [  295.074   357.658    13.164     0.031]
 [  753.546   379.476    14.096     0.065]
 [  940.446   461.984    12.265     0.012]
 [  515.882   508.629    11.082     0.005]
 [  355.176   583.665    13.403     0.037]
 [  952.449   599.439    14.264     0.071]
 [  389.024   681.497    14.431     0.09 ]
 [   52.301   703.531    10.301     0.003]
 [  748.465   705.381    13.275     0.033]
 [  235.315   705.592    13.409     0.037]
 [   52.876   708.867    10.28      0.003]
 [  385.6  

In [22]:
np.savetxt('result_calib.txt', mlist, fmt='%.3f')

### 演習3  

演習2で処理をしたtarget2のフレーム(例えばidf0020.fits)で測光をしてみましょう。  
target2のフレームには標準星は写っていません。しかし、target1と近い時刻に観測をしているので、target1と同じ較正値が使えるものとしてください。



